In [ ]:
import numpy as np

def log_cosh_loss(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=np.float64), np.array(y_pred, dtype=np.float64)
    
    # 動態縮放因子，僅根據 y_true 計算，防止溢位
    data_range = np.max(np.abs(y_true))
    if data_range > 1000:
        scaling_factor = 1e-5  # 進一步縮小縮放因子
    elif data_range > 10:
        scaling_factor = 1e-3
    else:
        scaling_factor = 1.0
    
    def _log_cosh(x):
        # 使用數值穩定的公式來避免溢位
        return np.where(np.abs(x) > 20, np.abs(x) - np.log(2), np.log(np.cosh(x)))
    
    # 使用縮放後的差異值來計算
    loss = _log_cosh(scaling_factor * (y_pred - y_true))
    
    # 將損失除以縮放因子以保持量級
    return np.mean(loss) / scaling_factor

A = []